In [5]:
%pip install beautifulsoup4 requests pandas joblib -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
from joblib import Parallel, delayed

In [3]:
BASE_URL = 'https://myanimelist.net/topanime.php?limit='
res = requests.get(F'{BASE_URL}0')
soup = bs(res.text, 'html.parser')

In [49]:
animes = []
def get_anime_values(tr):
    tds = tr.find_all('td')
    position = int(tds[0].text.strip()) if '-' not in tds[0].text.strip() else np.nan
    title = tds[1].find_all('h3')[0].text.strip()
    info = tds[1].find('div', {'class': 'information di-ib mt4'}).text.strip().split('\n')
    info = [i.strip() for i in info]
    episodes = info[0]
    release_date = info[1]
    members = int(info[2].replace(',', '').split()[0])
    score = float(tds[2].text.strip()) if 'N/A' not in tds[2].text else 0
    details = tr.find_all('a')[0].get('href').strip()
    
    return [position, title, episodes, release_date, members, score, details]

In [50]:
animes = Parallel(n_jobs=1, verbose=1)(delayed(get_anime_values)(tr) for tr in soup.find_all('tr')[1:])
animes

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


[[951,
  'Inu-Ou',
  'Movie (1 eps)',
  'May 2022 - May 2022',
  13720,
  7.78,
  'https://myanimelist.net/anime/39938/Inu-Ou'],
 [952,
  'Jin-Rou',
  'Movie (1 eps)',
  'Jun 2000 - Jun 2000',
  158165,
  7.78,
  'https://myanimelist.net/anime/570/Jin-Rou'],
 [953,
  'Kage no Jitsuryokusha ni Naritakute!',
  'TV (20 eps)',
  'Oct 2022 -',
  193985,
  7.79,
  'https://myanimelist.net/anime/48316/Kage_no_Jitsuryokusha_ni_Naritakute'],
 [954,
  'BanG Dream! Garupa☆Pico',
  'ONA (26 eps)',
  'Jul 2018 - Dec 2018',
  20179,
  7.77,
  'https://myanimelist.net/anime/37873/BanG_Dream_Garupa☆Pico'],
 [955,
  'Cardcaptor Sakura: Clear Card-hen Prologue - Sakura to Futatsu no Kuma',
  'OVA (1 eps)',
  'Sep 2017 - Sep 2017',
  39657,
  7.77,
  'https://myanimelist.net/anime/35320/Cardcaptor_Sakura__Clear_Card-hen_Prologue_-_Sakura_to_Futatsu_no_Kuma'],
 [956,
  'Colorful (Movie)',
  'Movie (1 eps)',
  'Aug 2010 - Aug 2010',
  182913,
  7.77,
  'https://myanimelist.net/anime/8142/Colorful_Movie'],


In [121]:
%%time
RANGE_LIMIT = 100000 # 100000 is the max limit
animes = []
for i in range(0, RANGE_LIMIT, 50):
    res = requests.get(F'{BASE_URL}{i}')
    soup = bs(res.text, 'html.parser')
    if r"This page doesn't exist." in soup.text:
        print('Finished')
        break
    print(f'Getting data from {i} to {i+50}')
    animes.extend(get_anime_values(tr) for tr in soup.find_all('tr')[1:])

Getting data from 23000 to 23050
Getting data from 23050 to 23100
Finished
CPU times: total: 438 ms
Wall time: 4.19 s


In [111]:
df = pd.DataFrame(animes, columns=['position', 'title', 'episodes', 'release_date', 'members', 'score', 'details'])
df.drop_duplicates(subset=['title', 'episodes', 'release_date'], inplace=True, ignore_index=True)
df.sample(5)

,position,title,episodes,release_date,members,score,details
1143,1145.0,BanG Dream! Movie: Episode of Roselia - I: Yak...,Movie (1 eps),Apr 2021 - Apr 2021,8056,7.69,https://myanimelist.net/anime/41780/BanG_Dream...
2754,2756.0,Break Blade,TV (12 eps),Apr 2014 - Jun 2014,89709,7.25,https://myanimelist.net/anime/22433/Break_Blade
4248,4250.0,Furusato Saisei: Nippon no Mukashibanashi,TV (258 eps),Apr 2012 - Mar 2017,10478,6.93,https://myanimelist.net/anime/13163/Furusato_S...
19267,19278.0,Inside,Music (1 eps),Apr 2022 - Apr 2022,96,0.00,https://myanimelist.net/anime/51767/Inside
21492,NaN,Joshikousei to Mahou no Note,ONA (? eps),2022 -,876,0.00,https://myanimelist.net/anime/49851/Joshikouse...


In [117]:
df[df.title.str.contains(',')].shape

(357, 7)

In [113]:
df[df.title.str.contains(';')].shape

(18, 7)

In [118]:
df.title = df.title.str.replace(',', '')

In [119]:
%%time
df.to_csv('AllAnimes.csv', index=False)

CPU times: total: 328 ms
Wall time: 476 ms
